In [17]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Map token names to contract addresses
token_address_map = {
    'rETH': '0xae78736Cd615f374D3085123A210448E74Fc6393'
    # fill with rest
}

# Map market name to "Pool" contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './AAVE_Pool_ABI.json'),
    'COMPOUND': ('0xA17581A9E3356d9A858b789D68B4d866e593aE94', './Compound_ABI.json')
    # fill with rest
}

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/7e4f5238262543919688a59d0fef4a1d'))

In [18]:
# Set up contract address and ABI for each market
contract_address, abi_filepath = contract_address_abi_map['COMPOUND']

# Web3 setup
with open(abi_filepath) as f:
    abi = json.load(f)

smart_contract = w3.eth.contract(address=contract_address, abi=abi)

In [19]:
# Find block number closest to timestamp with binary search
def find_block_by_timestamp(target_timestamp):
    low, high = 0, w3.eth.block_number
    while low < high:
        mid = (low + high) // 2
        mid_block_timestamp = w3.eth.get_block(mid).timestamp
        if mid_block_timestamp < target_timestamp:
            low = mid+1
        else:
            high = mid
    return low


# Helper to convert block number to readable timestamp string
def get_block_timestamp(block_num):
    block = w3.eth.get_block(block_num)
    timestamp = block.timestamp
    
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


In [20]:
# Get timestamp of timeframe provided
timeframe=90
current_time = datetime.now()
start_time = current_time - timedelta(days=timeframe)
st_timestamp = int(start_time.timestamp())

# Find the block associated with our start_time timestamp
st_block = find_block_by_timestamp(st_timestamp)

In [35]:
supplies = smart_contract.events.SupplyCollateral().get_logs(fromBlock=st_block,
                                                    toBlock='latest',
                                                    argument_filters={'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393'})

borrows = smart_contract.events.BuyCollateral().get_logs(fromBlock=st_block,
                                                    toBlock='latest',
                                                    argument_filters={'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393'})

transfers = smart_contract.events.TransferCollateral().get_logs(fromBlock=st_block,
                                                    toBlock='latest',
                                                    argument_filters={'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393'}       
                                                    )
# Consider simplifying supply_list.. don't know what information future markets will return, 
# ... Will most likely trim this down to: Reserve, User, Amount (in token, not Wei), Timestamp, and block number
# supply_list = [
#     {
#         'Reserve': event['args']['reserve'],
#         'User (Wallet Address)': event['args']['user'],
#         'Amount (Wei)': event['args']['amount'],
#         'Amount (rETH)': event['args']['amount']/10**18,
#         'Timestamp': get_block_timestamp(event['blockNumber']), # This particular line results in an aditional API call
#         'LogIndex': event['logIndex'],
#         'TransactionIndex': event['transactionIndex'],
#         'TransactionHash': event['transactionHash'].hex(),
#         'BlockHash': event['blockHash'].hex(),
#         'BlockNumber': event['blockNumber']
#     }
#     for event in supplies
# ]
supplies

(AttributeDict({'args': AttributeDict({'from': '0x121E1573B7C6c15Aa4C036d54BFd2974Cf38a163',
   'dst': '0x121E1573B7C6c15Aa4C036d54BFd2974Cf38a163',
   'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393',
   'amount': 8112001731285519}),
  'event': 'SupplyCollateral',
  'logIndex': 315,
  'transactionIndex': 127,
  'transactionHash': HexBytes('0x81d2e9db87affcac1f3e7f9bf16bdf531298b3c80c8de4cdd7e747b03ca29481'),
  'address': '0xA17581A9E3356d9A858b789D68B4d866e593aE94',
  'blockHash': HexBytes('0xc3f1143d8d57de487c3c87b70910b4f1b42e611d3b190f684093b5eacce06bc9'),
  'blockNumber': 18911780}),
 AttributeDict({'args': AttributeDict({'from': '0xA490A0346808dDA91aea6698cB19e4697D9fc5cc',
   'dst': '0xA490A0346808dDA91aea6698cB19e4697D9fc5cc',
   'asset': '0xae78736Cd615f374D3085123A210448E74Fc6393',
   'amount': 45469639104902929}),
  'event': 'SupplyCollateral',
  'logIndex': 255,
  'transactionIndex': 153,
  'transactionHash': HexBytes('0xb3641701dc283c565ccc2ef6e5c8a3139810c7fb5654d474

In [31]:
rETH_info = smart_contract.functions.totalsCollateral('0xae78736Cd615f374D3085123A210448E74Fc6393').call()
# Amount of rETH used for historical supply
rETH_info[0] / 10**18

145.01688768165906

In [26]:
rETH_reserves = smart_contract.functions.getCollateralReserves('0xae78736Cd615f374D3085123A210448E74Fc6393').call()
rETH_reserves

0

In [23]:
total_supply = smart_contract.functions.totalSupply().call()
total_supply

45063955125927280750277

In [24]:
total_borrow = smart_contract.functions.totalBorrow().call()
total_borrow

33928548142654006268517

In [32]:
supply_rate = smart_contract.functions.getSupplyRate().call()

Web3ValidationError: 
Could not identify the intended function with name `getSupplyRate`, positional arguments with type(s) `` and keyword arguments with type(s) `{}`.
Found 1 function(s) with the name `getSupplyRate`: ['getSupplyRate(uint256)']
Function invocation failed due to improper number of arguments.